In [437]:
import pandas as pd
import numpy as np

In [438]:
# Load data, 1001
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [439]:
# Load data, 90
#pwt1001 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

In [457]:
# Filter and select relevant columns, 1001
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'country', 'rgdpna', 'rkna', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [458]:
# Filter and select relevant columns, 90
#data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'country', 'rgdpna', 'rkna', 'emp','avh', 'labsh','rtfpna']]
#data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2020)].dropna()

In [459]:
# Calculate additional columns
#data['L'] = data['emp'] * data['avh']  # L
data['L'] = data['emp'] # L

data['ln_y'] = np.log(data['rgdpna'] / data['L']) # ln(Y/L)
#data['ln_A'] = np.log(data['rtfpna']) # A
data['ln_k'] = np.log(data['rkna'] / data['L'])  # ln(K/L)
data['alpha'] = 1 - data['labsh']  # alpha
data['alpha_x_ln_k'] = data['alpha'] * data['ln_k']  # alpha * ln(K/L)

In [460]:
# Order by year
data = data.sort_values('year')

In [478]:
# Group by country
grouped_data = data.groupby('country')

In [479]:
# Calculate growth rates, etc.
data['g_rate'] = (grouped_data['ln_y'].diff() * 100)  # Growth rate of GDP 
data['cap_d'] = data['alpha'] * (grouped_data['ln_k'].diff() * 100)  # Capital Deepening
#data['cap_d'] = (grouped_data['alpha_x_ln_k'].diff() * 100)  # Capital Deepening

data['tfp_g'] = data['g_rate'] - data['cap_d']  # TFP Growth
##data['tfp_g'] = (grouped_data['ln_A'].diff() * 100)  # TFP Growth

#data['tfp_s'] = data['tfp_g'] / data['g_rate']  # TFP Share
#data['cap_s'] = data['cap_d'] / data['g_rate']  # Capital Share

In [480]:
# Remove missing values
data = data.dropna()

In [481]:
# Calculate summary statistics
summary = data.groupby('country').agg({'g_rate': 'mean','tfp_g': 'mean','cap_d': 'mean'})

#'tfp_s': 'mean','cap_s': 'mean'

In [482]:
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g_rate']
summary['TFP Growth'] = summary['tfp_g']
summary['Capital Deepening'] = summary['cap_d']

#summary['TFP Share'] = summary['tfp_s']
#summary['Capital Share'] = summary['cap_s']
summary['TFP Share'] = summary['TFP Growth'] /summary['Growth Rate']
summary['Capital Share'] = summary['Capital Deepening'] / summary['Growth Rate']

In [483]:
# Caluculate average
summary.loc['Average'] = summary.mean()

In [484]:
# Print output
summary = summary.iloc[:, -5:]
table = summary.to_string(index = True)
lines = table.split('\n')
table_with_border = '\n'.join(lines[:-1]) + '\n' + '-' * len(lines[-1]) + '\n' + lines[-1]

print(table_with_border)

                Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
country                                                                             
Canada             0.899278    0.271214           0.628064   0.301591       0.698409
France             0.831355    0.264441           0.566914   0.318084       0.681916
Germany            0.653930    0.234803           0.419127   0.359065       0.640935
Italy             -0.076514   -0.590259           0.513745   7.714423      -6.714423
Japan              0.613519    0.034902           0.578617   0.056888       0.943112
United Kingdom     0.995636    0.452612           0.543024   0.454596       0.545404
United States      1.506967    0.733937           0.773030   0.487029       0.512971
------------------------------------------------------------------------------------
Average            0.774882    0.200236           0.574646   1.384525      -0.384525


In [485]:
# Comparison
goal = pd.read_excel("/Users/hamadawataru/Desktop/goal.xlsx")  # Excel file load
goal = goal.set_index(goal.columns[0])  # First column as row labels
table2 = goal.to_string(index = True)  # To table
print(table2)

correlation = summary.corrwith(goal)
mean_corr = round(np.mean(correlation), 6)
print(correlation)
print(f"Average: {mean_corr}")

                Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
country                                                                             
Canada                 1.57        0.95               0.63       0.60           0.40
France                 2.50        1.54               0.95       0.62           0.38
Germany                3.09        1.96               1.12       0.64           0.36
Italy                  4.04        2.10               1.94       0.52           0.48
Japan                  3.28        2.73               0.56       0.83           0.17
United Kingdom         1.90        1.31               0.58       0.69           0.31
United States          1.89        1.09               0.80       0.58           0.42
Average                2.41        1.61               0.80       0.68           0.32
Growth Rate         -0.852791
TFP Growth          -0.619874
Capital Deepening   -0.385618
TFP Share           -0.560489
Capital Share       -0.560489


In [486]:
# Equal to...?
summary['TFP Growth + Capital Deepening'] = summary['TFP Growth'] + summary['Capital Deepening']
summary['TFP Share + Capital Share'] = summary['TFP Share'] + summary['Capital Share']

In [487]:
print(summary.loc[:, ['Growth Rate','TFP Growth + Capital Deepening']])
print(summary.loc[:, ['TFP Share + Capital Share']])

                Growth Rate  TFP Growth + Capital Deepening
country                                                    
Canada             0.899278                        0.899278
France             0.831355                        0.831355
Germany            0.653930                        0.653930
Italy             -0.076514                       -0.076514
Japan              0.613519                        0.613519
United Kingdom     0.995636                        0.995636
United States      1.506967                        1.506967
Average            0.774882                        0.774882
                TFP Share + Capital Share
country                                  
Canada                                1.0
France                                1.0
Germany                               1.0
Italy                                 1.0
Japan                                 1.0
United Kingdom                        1.0
United States                         1.0
Average               